In [4]:
# import dependencies
import os
import io
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime
from pytz import timezone
import pandas as pd
import FoodTools

In [5]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [7]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [8]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [9]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)
print('Length: ', len(meal_type))

['Breakfast', 'Lunch', 'Dinner']
Length:  3


In [10]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)
print('Length: ', len(food_list))

['Belgian Waffles', 'Cage Free Scrambled Eggs', "Egg O'Bagel", 'Hard-boiled Cage Free Eggs', 'Mexican Chorizo Sausage', 'Savory Cheese Grits', 'Soyrizo Tofu Scramble', 'Spicy Gold Potatoes', 'Texas French Toast', 'Lemon Poppyseed Muffin', 'Muffin Orange Cranberry', 'Raspberry Cheese Ring', 'Vegan Peanut Granola', 'Bell Peppers', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cheddar Cheese', 'Feta Cheese', 'Fresh Organic Spinach', 'Fresh Tomato', 'Jalapeno', 'Onions', 'Oven Roasted Turkey', 'Sliced Olived', 'Smoked Ham', 'Chicken Ortega Soup', 'White Bean with Kale Soup', 'Oven Roasted Chicken Thigh', 'Oven Roasted Herbed Potatoes', 'Steamed Rice', 'Steamed Seasonal Vegetables', 'Sun Dried Tomato Crusted Chicken Breast', 'Cheese Pizza', 'Roasted Vegetable Pizza', 'Portobella Mushroom Cheese Steak', 'Lemon Bars', 'Peanut Butter Brownie', 'Atomic Cheese Sauce', 'BAR Boardwalk Burgers', 'Bacon', 'Bread Bun', 'Burger Beef', 'Burger Turkey', 'Chili with Beef', 'Onion Rings', 'Sauteed Mushroom

In [11]:
# In case of dupliate food name, the list should be converted to a set
# sort
food_list = sorted(list(set(food_list)))
print(food_list)
print('Sorted length: ', len(food_list))

['Atomic Cheese Sauce', 'BAR Boardwalk Burgers', 'BAR Street Taco', 'BBQ Ginger Soy Chicken', 'BBQ Tofu Chili Garlic', 'BLT Pizza', 'Bacon', 'Belgian Waffles', 'Bell Peppers', 'Bread Bun', 'Burger Beef', 'Burger Turkey', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cage Free Scrambled Eggs', 'Cheddar Cheese', 'Cheese Pizza', 'Chicken Ortega Soup', 'Chili Soy Slaw', 'Chili with Beef', 'Condiments', "Egg O'Bagel", 'Feta Cheese', 'Fresh Organic Spinach', 'Fresh Tomato', 'Hard-boiled Cage Free Eggs', 'Herbed Pork Chop', 'Italian Sausage Sandwich', 'Jalapeno', 'Korean BBQ Beef', 'Lemon Bars', 'Lemon Poppyseed Muffin', 'Mexican Chorizo Sausage', 'Muffin Orange Cranberry', 'Onion Rings', 'Onions', 'Oven Roasted Chicken Thigh', 'Oven Roasted Herbed Potatoes', 'Oven Roasted Turkey', 'Peanut Butter Brownie', 'Portobella Mushroom Cheese Steak', 'Quinoa with Butternut Squash and Kale', 'Raspberry Cheese Ring', 'Roasted Vegetable Pizza', 'Salsa Roja', 'Sauteed Mushrooms', 'Sauteed Seasonal Vegetabl

In [12]:
# preference list, labeled by myself
# pref_1 = [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0 ]
# pref_2 = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
# pref_3 = [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
# pref_4 = [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
# pref_5 = [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
pref = []
for i in food_list:
    print(i)
    preference = int(input())
    if preference != 0 and preference != 1:
        print('Incrorrect input')
        break
    pref.append(preference)
print(pref)
print('Preference length: ', len(pref))

Atomic Cheese Sauce
0
BAR Boardwalk Burgers
1
BAR Street Taco
0
BBQ Ginger Soy Chicken
1
BBQ Tofu Chili Garlic
0
BLT Pizza
0
Bacon
1
Belgian Waffles
0
Bell Peppers
0
Bread Bun
0
Burger Beef
1
Burger Turkey
0
Cage Free Eggs
0
Cage Free Omelette Bar
0
Cage Free Scrambled Eggs
1
Cheddar Cheese
0
Cheese Pizza
1
Chicken Ortega Soup
1
Chili Soy Slaw
0
Chili with Beef
1
Condiments
0
Egg O'Bagel
0
Feta Cheese
0
Fresh Organic Spinach
0
Fresh Tomato
0
Hard-boiled Cage Free Eggs
0
Herbed Pork Chop
0
Italian Sausage Sandwich
0
Jalapeno
0
Korean BBQ Beef
1
Lemon Bars
0
Lemon Poppyseed Muffin
0
Mexican Chorizo Sausage
0
Muffin Orange Cranberry
0
Onion Rings
0
Onions
0
Oven Roasted Chicken Thigh
0
Oven Roasted Herbed Potatoes
0
Oven Roasted Turkey
0
Peanut Butter Brownie
0
Portobella Mushroom Cheese Steak
0
Quinoa with Butternut Squash and Kale
0
Raspberry Cheese Ring
0
Roasted Vegetable Pizza
0
Salsa Roja
0
Sauteed Mushrooms
0
Sauteed Seasonal Vegetable
0
Savory Cheese Grits
0
Sliced Olived
0
Smoked

In [13]:
# timezone setting
us_pacific = timezone('US/Pacific')
time = datetime.now(us_pacific)
us_time = time.strftime('%Y-%m-%d')
print('Current US/Pacific time: ', us_time)

Current US/Pacific time:  2018-09-20


In [14]:
# write the food list to a csv file
path = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/{}.csv'.format(us_time)
print('Current path: ', path)
path_2 = '/Users/7w0r4ng3s/Desktop/menu_scraping/data/'

Current path:  /Users/7w0r4ng3s/Desktop/menu_scraping/data/2018-09-20.csv


In [15]:
def write_data():
    with open(path, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for key, val in zip(food_list, pref):
            writer.writerow([key, val])
    print('write_csv: COMPLETED')
    # add index and column names
    df = pd.read_csv(path, names=['food', 'pref'])
    df.index.names = ['index']
    df.to_csv(path)
    print('add_column_name: COMPLETED')
    
write_data()

write_csv: COMPLETED
add_column_name: COMPLETED


In [16]:
# def merge_data():
#     # TODO: Figure out a way to get rid of the unnamed: 0 column
#     # TODO: Modify merge_data() so that new data can be append to data.csv
#     files = [f for f in os.listdir('.') if os.path.isfile(f)]

#     merged = []

#     for f in files:
#         filename, ext = os.path.splitext(f)
#         if ext == '.csv':
#             read = pd.read_csv(f)
#             merged.append(read)

#     result = pd.concat(merged)
#     result.to_csv('data.csv')
    
# merge_data()

In [17]:
def append_data():
    df1 = pd.read_csv('Data.csv', index_col='index')
    df1.index.names = ['index']
    df2 = pd.read_csv(path, index_col='index')
    df3 = df1.append(df2).reset_index()
    df3 = df3.drop('index', 1).sort_values('food').reset_index().drop('index', 1)
    df3.index.names = ['index']
    df3.to_csv('Data.csv')
    
append_data()

In [18]:
FoodTools.search()

Food name: chili
Do you mean:
1 BBQ Tofu Chili Garlic
2 Chili con Carne
3 Szechuan Sweet Chili Sauce

Enter the index: 3
--------------------------------------------------------------------------------
Out of 2 time(s), you like [Szechuan Sweet Chili Sauce] 2 time(s).
--------------------------------------------------------------------------------


1.0